In [ ]:
import gen_data
import impute_data
import matplotlib.pyplot as plt
import numpy as np
from utils import binary_sampler
import data_utils

In [ ]:
#import zip dataset as numpy matrix 
data = data_utils.load_training_data('dataset/ecg_data/1 NSR.zip','.mat')
data.shape


In [ ]:
# Generate 5 synthetic ECG data frames
# hyperparameters: 1. data file 2. data type (.mat or .csv) 3. number of synthetic sequences to be generated 
# 4. at what epochs to save model (< epoch) 5. training epoch 6. model output dir
synthesized_data = gen_data.gen_data_GAN(data = data, 
                 data_type = '.mat', 
                 num_seq = 5, 
                 model_chkpoint = 2,
                 num_epochs = 10, 
                 out_dir = 'models/')

In [ ]:
synthesized_data.shape

In [ ]:
#Suppose user has corrected some parts and emulated missing data in one of the plots
correction_by_user = np.array([synthesized_data[0,:]])
correction_by_user.shape
#emulating missing data by user:
data_m = binary_sampler(1-0.3, correction_by_user.shape[0], correction_by_user.shape[1])
inp_data = correction_by_user.copy()
inp_data[data_m == 0] = np.nan
inp_data = inp_data.flatten()
fig, axes = plt.subplots(1,2, figsize=((14,8)))
axes[0].plot(synthesized_data[0,:])
axes[0].set_title('Generated data')
axes[1].plot(inp_data)
axes[1].set_title('User corrected data')

In [ ]:
# Generate several suggestions for corrected (imputed) data as wanted by user
# hyperparameters: 1. data file 2. data type (.mat or .csv) 3. vector to be imputed
#4. missing data rate during training 5. batch size 6. hint rate 7. alpha 8. no. of epochs

imputed_data = impute_data.impute_data(orig_data = data,
                data_type = '.mat',
                inp_data = inp_data,
                miss_rate=0.3,
                batch_size = 128,
                hint_rate= 0.9,
                alpha = 100,
                iterations = 10000)

In [ ]:
#Check out one of the suggestions for imputation
fig, axes = plt.subplots(1,3, figsize=((14,8)))
axes[0].plot(synthesized_data[0,:])
axes[0].set_title('Generated data')
axes[1].plot(inp_data)
axes[1].set_title('User corrected data')
axes[2].plot(imputed_data[:,0])
axes[2].set_title('Imputed data by GAN') #a little imperfect due to normalization errors but will fix later